In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from helper import TelecomHelper
from clean_telecom_df import CleanTelecomData

In [4]:
CSV_PATH = "../data/data.xlsx"

In [5]:
Helper = TelecomHelper()

In [6]:
df = Helper.read_csv(CSV_PATH, missing_values=["n/a", "na", "undefined"])

file read as csv


In [7]:
pd.set_option('max_column', None)
pd.options.display.float_format = "{:.6f}".format


In [8]:
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.000000,2019-04-25 14:35:31,662.000000,1823652.000000,208201448079117.000000,33664962239.000000,35521209507511.000000,9164566995485190,42.000000,5.000000,23.000000,44.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,37624.000000,38787.000000,1823652892.000000,Samsung,Samsung Galaxy A5 Sm-A520F,nan,nan,nan,nan,nan,nan,213.000000,214.000000,1545765.000000,24420.000000,1634479.000000,1271433.000000,3563542.000000,137762.000000,15854611.000000,2501332.000000,8198936.000000,9656251.000000,278082303.000000,14344150.000000,171744450.000000,8814393.000000,36749741.000000,308879636.000000
1,13114483482878900224,2019-04-09 13:04:04,235.000000,2019-04-25 08:15:48,606.000000,1365104.000000,208201909211140.000000,33681854413.000000,35794009006359.000000,L77566A,65.000000,5.000000,16.000000,26.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,168.000000,3560.000000,1365104371.000000,Samsung,Samsung Galaxy J5 (Sm-J530),nan,nan,nan,nan,nan,nan,971.000000,1022.000000,1926113.000000,7165.000000,3493924.000000,920172.000000,629046.000000,308339.000000,20247395.000000,19111729.000000,18338413.000000,17227132.000000,608750074.000000,1170709.000000,526904238.000000,15055145.000000,53800391.000000,653384965.000000
2,13114483484080500736,2019-04-09 17:42:11,1.000000,2019-04-25 11:58:13,652.000000,1361762.000000,208200314458056.000000,33760627129.000000,35281510359387.000000,D42335A,nan,nan,6.000000,9.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,0.000000,0.000000,1361762651.000000,Samsung,Samsung Galaxy A8 (2018),nan,nan,nan,nan,nan,nan,751.000000,695.000000,1684053.000000,42224.000000,8535055.000000,1694064.000000,2690151.000000,672973.000000,19725661.000000,14699576.000000,17587794.000000,6163408.000000,229584621.000000,395630.000000,410692588.000000,4215763.000000,27883638.000000,279807335.000000
3,13114483485442799616,2019-04-10 00:31:25,486.000000,2019-04-25 07:36:35,171.000000,1321509.000000,208201402342131.000000,33750343200.000000,35356610164913.000000,T21824A,nan,nan,44.000000,44.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,3330.000000,37882.000000,1321509685.000000,undefined,undefined,nan,nan,nan,nan,nan,nan,17.000000,207.000000,644121.000000,13372.000000,9023734.000000,2788027.000000,1439754.000000,631229.000000,21388122.000000,15146643.000000,13994646.000000,1097942.000000,799538153.000000,10849722.000000,749039933.000000,12797283.000000,43324218.000000,846028530.000000
4,13114483499480700928,2019-04-12 20:10:23,565.000000,2019-04-25 10:40:32,954.000000,1089009.000000,208201401415120.000000,33699795932.000000,35407009745539.000000,D88865A,nan,nan,6.000000,9.000000,nan,nan,100.0

In [9]:
def convert_labels(df):
    df.columns = [column.replace(' ', '_').lower() for column in df.columns]
    return df
df = convert_labels(df)    

In [10]:
class OverViewAnalysis:
    
    def __init__(self, df):
        
        self.df = df
    
    
    def read_head(self, top=5):
        return self.df.head(top)
    
    # returning the number of rows columns and column information
    def get_info(self):
        row_count, col_count = self.df.shape
    
        print(f"Number of rows: {row_count}")
        print(f"Number of columns: {col_count}")
        print("================================")

        return (row_count, col_count), self.df.info()
    
    # gets number of distnict values in a given coumn
    def get_count(self, column_name):
        return self.df[column_name].value_counts()
    
    # getting the null count for every column
    def get_null_count(self, column_name):
        print("Null values count")
        print(self.df.isnull().sum())
        return self.df.isnull().sum()
    
    # getting the percentage of missing values
    def get_percent_missing(self):
        Helper = TelecomHelper()
        
        percent_missing = Helper.percent_missing(self.df)
        
        null_percent_df = pd.DataFrame(columns = ['column', 'null_percent'])
        columns = self.df.columns.values.tolist()
        
        null_percent_df['column'] = columns
        null_percent_df['null_percent'] = null_percent_df['column'].map(lambda x: Helper.percent_missing_for_col(self.df, x))
        
        
        return null_percent_df.sort_values(by=['null_percent'], ascending = False), percent_missing
    
    
    # returning the top handseet
    def top_handset_type(self, top=5):
        
        return self.df['handset_type'].value_counts().head(top)
    
    # returning the top handset manufactruer
    def top_manufacturer(self, top=5):
        
        return self.df['handset_manufacturer'].value_counts().head(top)
    
    # top handset by manfuctruer
    def top_handset_by_manufacturer(self, manufacturer, top=5):
        
        return df.groupby('handset_manufacturer')['handset_type'].value_counts()[manufacturer].head(top)
 

In [11]:
overViewAnal = OverViewAnalysis(df)


In [12]:
overViewAnal.read_head()

,bearer_id,start,start_ms,end,end_ms,dur._(ms),imsi,msisdn/number,imei,last_location_name,avg_rtt_dl_(ms),avg_rtt_ul_(ms),avg_bearer_tp_dl_(kbps),avg_bearer_tp_ul_(kbps),tcp_dl_retrans._vol_(bytes),tcp_ul_retrans._vol_(bytes),dl_tp_<_50_kbps_(%),50_kbps_<_dl_tp_<_250_kbps_(%),250_kbps_<_dl_tp_<_1_mbps_(%),dl_tp_>_1_mbps_(%),ul_tp_<_10_kbps_(%),10_kbps_<_ul_tp_<_50_kbps_(%),50_kbps_<_ul_tp_<_300_kbps_(%),ul_tp_>_300_kbps_(%),http_dl_(bytes),http_ul_(bytes),activity_duration_dl_(ms),activity_duration_ul_(ms),dur._(ms).1,handset_manufacturer,handset_type,nb_of_sec_with_125000b_<_vol_dl,nb_of_sec_with_1250b_<_vol_ul_<_6250b,nb_of_sec_with_31250b_<_vol_dl_<_125000b,nb_of_sec_with_37500b_<_vol_ul,nb_of_sec_with_6250b_<_vol_dl_<_31250b,nb_of_sec_with_6250b_<_vol_ul_<_37500b,nb_of_sec_with_vol_dl_<_6250b,nb_of_sec_with_vol_ul_<_1250b,social_media_dl_(bytes),social_media_ul_(bytes),google_dl_(bytes),google_ul_(bytes),email_dl_(bytes),email_ul_(bytes),youtube_dl_(bytes),youtube_ul_(bytes),netflix_dl_(bytes),netflix_ul_(bytes),gaming_dl_(bytes),gaming_ul_(bytes),other_dl_(bytes),other_ul_(bytes),total_ul_(bytes),total_dl_(bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.000000,2019-04-25 14:35:31,662.000000,1823652.000000,208201448079117.000000,33664962239.000000,35521209507511.000000,9164566995485190,42.000000,5.000000,23.000000,44.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,37624.000000,38787.000000,1823652892.000000,Samsung,Samsung Galaxy A5 Sm-A520F,nan,nan,nan,nan,nan,nan,213.000000,214.000000,1545765.000000,24420.000000,1634479.000000,1271433.000000,3563542.000000,137762.000000,15854611.000000,2501332.000000,8198936.000000,9656251.000000,278082303.000000,14344150.000000,171744450.000000,8814393.000000,36749741.000000,308879636.000000
1,13114483482878900224,2019-04-09 13:04:04,235.000000,2019-04-25 08:15:48,606.000000,1365104.000000,208201909211140.000000,33681854413.000000,35794009006359.000000,L77566A,65.000000,5.000000,16.000000,26.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,168.000000,3560.000000,1365104371.000000,Samsung,Samsung Galaxy J5 (Sm-J530),nan,nan,nan,nan,nan,nan,971.000000,1022.000000,1926113.000000,7165.000000,3493924.000000,920172.000000,629046.000000,308339.000000,20247395.000000,19111729.000000,18338413.000000,17227132.000000,608750074.000000,1170709.000000,526904238.000000,15055145.000000,53800391.000000,653384965.000000
2,13114483484080500736,2019-04-09 17:42:11,1.000000,2019-04-25 11:58:13,652.000000,1361762.000000,208200314458056.000000,33760627129.000000,35281510359387.000000,D42335A,nan,nan,6.000000,9.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,0.000000,0.000000,1361762651.000000,Samsung,Samsung Galaxy A8 (2018),nan,nan,nan,nan,nan,nan,751.000000,695.000000,1684053.000000,42224.000000,8535055.000000,1694064.000000,2690151.000000,672973.000000,19725661.000000,14699576.000000,17587794.000000,6163408.000000,229584621.000000,395630.000000,410692588.000000,4215763.000000,27883638.000000,279807335.000000
3,13114483485442799616,2019-04-10 00:31:25,486.000000,2019-04-25 07:36:35,171.000000,1321509.000000,208201402342131.000000,33750343200.000000,35356610164913.000000,T21824A,nan,nan,44.000000,44.000000,nan,nan,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,nan,nan,3330.000000,37882.000000,1321509685.000000,undefined,undefined,nan,nan,nan,nan,nan,nan,17.000000,207.000000,644121.000000,13372.000000,9023734.000000,2788027.000000,1439754.000000,631229.000000,21388122.000000,15146643.000000,13994646.000000,1097942.000000,799538153.000000,10849722.000000,749039933.000000,12797283.000000,43324218.000000,846028530.000000
4,13114483499480700928,2019-04-12 20:10:23,565.000000,2019-04-25 10:40:32,954.000000,1089009.000000,208201401415120.000000,33699795932.000000,35407009745539.000000,D88865A,nan,nan,6.000000,9.000000,nan,nan,100.0

In [13]:
overViewAnal.get_info()

Number of rows: 150001
Number of columns: 55
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   bearer_id                                 150001 non-null  object        
 1   start                                     150000 non-null  datetime64[ns]
 2   start_ms                                  150000 non-null  float64       
 3   end                                       150000 non-null  datetime64[ns]
 4   end_ms                                    150000 non-null  float64       
 5   dur._(ms)                                 150000 non-null  float64       
 6   imsi                                      149431 non-null  float64       
 7   msisdn/number                             148935 non-null  float64       
 8   imei                                      149429 

((150001, 55), None)

In [14]:
df.isnull().sum()

bearer_id                                        0
start                                            1
start_ms                                         1
end                                              1
end_ms                                           1
dur._(ms)                                        1
imsi                                           570
msisdn/number                                 1066
imei                                           572
last_location_name                            1153
avg_rtt_dl_(ms)                              27829
avg_rtt_ul_(ms)                              27812
avg_bearer_tp_dl_(kbps)                          1
avg_bearer_tp_ul_(kbps)                          1
tcp_dl_retrans._vol_(bytes)                  88146
tcp_ul_retrans._vol_(bytes)                  96649
dl_tp_<_50_kbps_(%)                            754
50_kbps_<_dl_tp_<_250_kbps_(%)                 754
250_kbps_<_dl_tp_<_1_mbps_(%)                  754
dl_tp_>_1_mbps_(%)             

In [15]:
null_percent_df, overall_missing = overViewAnal.get_percent_missing()
print(f"The overall missing perceentage is: {overall_missing}%")
print("=============================================")
null_percent_df

The overall missing perceentage is: 12.49%


,column,null_percent
34,nb_of_sec_with_37500b_<_vol_ul,86.840000
36,nb_of_sec_with_6250b_<_vol_ul_<_37500b,74.560000
31,nb_of_sec_with_125000b_<_vol_dl,65.020000
15,tcp_ul_retrans._vol_(bytes),64.430000
33,nb_of_sec_with_31250b_<_vol_dl_<_125000b,62.390000
32,nb_of_sec_with_1250b_<_vol_ul_<_6250b,61.930000
35,nb_of_sec_with_6250b_<_vol_dl_<_31250b,58.880000
14,tcp_dl_retrans._vol_(bytes),58.760000
25,http_ul_(bytes),54.540000
24,http_dl_(bytes),54.320000


In [16]:
cleanTelecomData = CleanTelecomData(df)

Automation in Action...!!!


In [17]:
cleaned_df = cleanTelecomData.convert_to_datetime(df)

cleaned_df = cleanTelecomData.drop_duplicate(cleaned_df)

cleaned_df = cleanTelecomData.drop_rows_with_null_values(cleaned_df, 1)

cleaned_df = cleanTelecomData.handle_missing_qantitative_data_with_mean(cleaned_df)

cleaned_df = cleanTelecomData.handle_missing_categorical_data_with(cleaned_df, "ffill")


overViewAnal_2 = OverViewAnalysis(cleaned_df)


In [18]:
cleaned_df.head()

,bearer_id,start,start_ms,end,end_ms,dur._(ms),imsi,msisdn/number,imei,last_location_name,avg_rtt_dl_(ms),avg_rtt_ul_(ms),avg_bearer_tp_dl_(kbps),avg_bearer_tp_ul_(kbps),tcp_dl_retrans._vol_(bytes),tcp_ul_retrans._vol_(bytes),dl_tp_<_50_kbps_(%),50_kbps_<_dl_tp_<_250_kbps_(%),250_kbps_<_dl_tp_<_1_mbps_(%),dl_tp_>_1_mbps_(%),ul_tp_<_10_kbps_(%),10_kbps_<_ul_tp_<_50_kbps_(%),50_kbps_<_ul_tp_<_300_kbps_(%),ul_tp_>_300_kbps_(%),http_dl_(bytes),http_ul_(bytes),activity_duration_dl_(ms),activity_duration_ul_(ms),dur._(ms).1,handset_manufacturer,handset_type,nb_of_sec_with_125000b_<_vol_dl,nb_of_sec_with_1250b_<_vol_ul_<_6250b,nb_of_sec_with_31250b_<_vol_dl_<_125000b,nb_of_sec_with_37500b_<_vol_ul,nb_of_sec_with_6250b_<_vol_dl_<_31250b,nb_of_sec_with_6250b_<_vol_ul_<_37500b,nb_of_sec_with_vol_dl_<_6250b,nb_of_sec_with_vol_ul_<_1250b,social_media_dl_(bytes),social_media_ul_(bytes),google_dl_(bytes),google_ul_(bytes),email_dl_(bytes),email_ul_(bytes),youtube_dl_(bytes),youtube_ul_(bytes),netflix_dl_(bytes),netflix_ul_(bytes),gaming_dl_(bytes),gaming_ul_(bytes),other_dl_(bytes),other_ul_(bytes),total_ul_(bytes),total_dl_(bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.000000,2019-04-25 14:35:31,662.000000,1823652.000000,208201448079117.000000,33664962239.000000,35521209507511.000000,9164566995485190,42.000000,5.000000,23.000000,44.000000,21052378.161588,772902.274842,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,117028627.917559,3296942.293956,37624.000000,38787.000000,1823652892.000000,Samsung,Samsung Galaxy A5 Sm-A520F,999.761617,344.663898,820.911894,150.524260,976.983998,142.876664,213.000000,214.000000,1545765.000000,24420.000000,1634479.000000,1271433.000000,3563542.000000,137762.000000,15854611.000000,2501332.000000,8198936.000000,9656251.000000,278082303.000000,14344150.000000,171744450.000000,8814393.000000,36749741.000000,308879636.000000
1,13114483482878900224,2019-04-09 13:04:04,235.000000,2019-04-25 08:15:48,606.000000,1365104.000000,208201909211140.000000,33681854413.000000,35794009006359.000000,L77566A,65.000000,5.000000,16.000000,26.000000,21052378.161588,772902.274842,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,117028627.917559,3296942.293956,168.000000,3560.000000,1365104371.000000,Samsung,Samsung Galaxy J5 (Sm-J530),999.761617,344.663898,820.911894,150.524260,976.983998,142.876664,971.000000,1022.000000,1926113.000000,7165.000000,3493924.000000,920172.000000,629046.000000,308339.000000,20247395.000000,19111729.000000,18338413.000000,17227132.000000,608750074.000000,1170709.000000,526904238.000000,15055145.000000,53800391.000000,653384965.000000
2,13114483484080500736,2019-04-09 17:42:11,1.000000,2019-04-25 11:58:13,652.000000,1361762.000000,208200314458056.000000,33760627129.000000,35281510359387.000000,D42335A,105.959783,17.633655,6.000000,9.000000,21052378.161588,772902.274842,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,117028627.917559,3296942.293956,0.000000,0.000000,1361762651.000000,Samsung,Samsung Galaxy A8 (2018),999.761617,344.663898,820.911894,150.524260,976.983998,142.876664,751.000000,695.000000,1684053.000000,42224.000000,8535055.000000,1694064.000000,2690151.000000,672973.000000,19725661.000000,14699576.000000,17587794.000000,6163408.000000,229584621.000000,395630.000000,410692588.000000,4215763.000000,27883638.000000,279807335.000000
3,13114483485442799616,2019-04-10 00:31:25,486.000000,2019-04-25 07:36:35,171.000000,1321509.000000,208201402342131.000000,33750343200.000000,35356610164913.000000,T21824A,105.959783,17.633655,44.000000,44.000000,21052378.161588,772902.274842,100.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,117028627.917559,3296942.293956,3330.000000,37882.000000,1321509685.000000,undefined,undefined,999.761617,344.663898,820.911894,150.524260,976.983998,142.876664,17.000000,207.000000,644121.000000,13372.000000,9023734.000000,2788027.000000,1439754.000000,631229.00

In [19]:
null_percent_df, overall_missing = overViewAnal_2.get_percent_missing()
print(f"The overall missing perceentage is: {overall_missing}%")
print("=============================================")

The overall missing perceentage is: 0.0%


In [20]:
# save the cleaned data to a csv file
cleaned_df.to_csv("../data/cleaned_data.csv")